# Notebook for the training of the Teresa Robot
## Importing all necessary tools

In [1]:
from src.gym_envs.RobotEnv import RobotEnv # Training environment
import numpy as np
import tensorflow.compat.v1 as tf
import roslibpy # API of ROS
from src.robots.Teresa import Teresa # This is the representation of Teresa Robot
from src.utils.training_tools import NB_STATES

tf.disable_v2_behavior()

Instructions for updating:
non-resource variables are not supported in the long term


2021-08-04 15:49:04,445  WARNING: From /home/saadubuntu/Documents/Teresa_robot_internship_2021/venv/lib/python3.6/site-packages/tensorflow_core/python/compat/v2_compat.py:68: disable_resource_variables (from tensorflow.python.ops.variable_scope) is deprecated and will be removed in a future version.
Instructions for updating:
non-resource variables are not supported in the long term


## Creating the connection with ROS

In [2]:
HOST = 'localhost'
PORT = 9090

client = roslibpy.Ros(host=HOST, port=PORT)


## Creating representation of the robot and introducing it in the Gym Environment

## Testing the environment

In [3]:
# client.connect()
client.run() # This run the main loop of ROS
teresa_controller = Teresa(client) # Robot API
env = RobotEnv(teresa_controller, client) # Training Environment

env.reset() # Restarting the environment to the initial state
import time
for i in range(2):
    state, reward, done, _ = env.step(np.random.randint(4))
    print(state)
    env.render()
    if done and reward:
        print("Body detected and centered")
        #env.reset()
    elif done:
        print("Face not detected. End of the episode")
        #env.reset()


2021-08-04 15:49:04,610     INFO: Connection to ROS MASTER ready.


2480
Body detected and centered
2364
Body detected and centered


## Controlling the Robot Manually

In [4]:
client.run()
teresa_controller = Teresa(client)
env = RobotEnv(teresa_controller, client)

env.reset()
finish = False

while not finish:
    movement = input('Enter a movement (0 Right, 1 Left, 2 Backward, 3 Forward, exit): ')
    if movement == 'exit':
        finish = True
        continue
    movement = int(movement)
    state, reward, done, _ = env.step(movement)
    if done and reward:
        print("Centered")
        #env.reset()
    env.render()

Enter a movement (0 Right, 1 Left, 2 Backward, 3 Forward, exit): exit


## Testing the filter haarcascade

In [5]:
import numpy as np
import cv2
from matplotlib import pyplot as plt

face_cascade = cv2.CascadeClassifier('Haarcascades/haarcascade_frontalface_default.xml')
eye_cascade = cv2.CascadeClassifier('Haarcascades/haarcascade_eye.xml')
body_cascade = cv2.CascadeClassifier('Haarcascades/haarcascade_fullbody.xml')

img = cv2.imread('env_observation')
gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

faces = face_cascade.detectMultiScale(gray, 1.2, 3)
body = body_cascade.detectMultiScale(gray,1.2,3)

if len(body)>0:
    print("body detected")
    x, y, w, h = body[0]
    cv2.rectangle(img, (x, y), (x+w, y+h), (25, 125, 225), 5)

print("ok")
for (x,y,w,h) in faces:
    cv2.rectangle(img,(x,y),(x+w,y+h),(255,0,0),2)
    roi_gray = gray[y:y+h, x:x+w]
    roi_color = img[y:y+h, x:x+w]
    eyes = eye_cascade.detectMultiScale(roi_gray)
    for (ex,ey,ew,eh) in eyes:
        cv2.rectangle(roi_color,(ex,ey),(ex+ew,ey+eh),(0,255,0),2)

print('ok')

cv2.imshow('img',img)
cv2.waitKey(0)
cv2.destroyAllWindows()

ok
ok


In [6]:
faces

()

## Necessary functions for the training (Neural Network Set up)

In [4]:
learning_rate = 0.01

state_size =  NB_STATES # 800 is the image size this maybe variable
action_size = 4
new_graph = tf.Graph()
## TRAINING Hyperparameters

initializer=tf.initializers.glorot_uniform()

def discount_correct_rewards(r, gamma=0.99):
  """ take 1D float array of rewards and compute discounted reward """
  discounted_r = np.zeros_like(r)
  running_add = 0
  for t in reversed(range(0, r.size)):
    #if r[t] != 0: running_add = 0 # reset the sum, since this was a game boundary (pong specific!)
    running_add = running_add * gamma + r[t]
    discounted_r[t] = running_add

  discounted_r -= discounted_r.mean()
  discounted_r /- discounted_r.std()
  return discounted_r

def discount_and_normalize_rewards(episode_rewards):
    discounted_episode_rewards = np.zeros_like(episode_rewards)
    cumulative = 0.0
    #print("len episode rewards",episode_rewards)
    for i in reversed(range(len(episode_rewards))):
        cumulative = cumulative * gamma + episode_rewards[i]
        #print("dans boucle",episode_rewards[i],"cyl",cumulative)
        discounted_episode_rewards[i] = cumulative
    
    mean = np.mean(discounted_episode_rewards)
    std = np.std(discounted_episode_rewards)
    if std :
        discounted_episode_rewards = (discounted_episode_rewards - mean) / (std)
    else:
        discounted_episode=[]
        discounted_episode_rewards[0] = np.array(mean)
        print("ATTTTTTTTTTTTTTTTTT")
    #print("dis",discounted_episode_rewards,"std",std)
    
    return discounted_episode_rewards

with tf.name_scope("inputs"):
    input_ = tf.placeholder(tf.float32, [None, state_size], name="input_")
    actions = tf.placeholder(tf.int32, [None, action_size], name="actions")
    discounted_episode_rewards_ = tf.placeholder(tf.float32, [None,], name="discounted_episode_rewards")
    
    # Add this placeholder for having this variable in tensorboard
    mean_reward_ = tf.placeholder(tf.float32 , name="mean_reward")

    with tf.name_scope("fc1"):
        fc1 = tf.layers.dense(input_ , 20, activation=tf.nn.relu,kernel_initializer=initializer)

    with tf.name_scope("fc2"):
        fc2 = tf.layers.dense(fc1, action_size,activation= tf.nn.relu, kernel_initializer=initializer)
    
    with tf.name_scope("fc3"):
        fc3 = tf.layers.dense(fc2, action_size, activation= None,kernel_initializer=initializer)

    with tf.name_scope("softmax"):
        action_distribution = tf.nn.softmax(fc3)

    with tf.name_scope("loss"):
        # tf.nn.softmax_cross_entropy_with_logits computes the cross entropy of the result after applying the softmax function
        # If you have single-class labels, where an object can only belong to one class, you might now consider using 
        # tf.nn.sparse_softmax_cross_entropy_with_logits so that you don't have to convert your labels to a dense one-hot array. 
        neg_log_prob = tf.nn.softmax_cross_entropy_with_logits_v2(logits = fc3, labels = actions)
        #loss = tf.nn.sparse_softmax_cross_entropy_with_logits (neg_log_prob * discounted_episode_rewards_)
        loss = tf.reduce_mean(neg_log_prob * discounted_episode_rewards_) 
        
    
    with tf.name_scope("train"):
        train_opt = tf.train.AdamOptimizer(learning_rate).minimize(loss)
# Setup TensorBoard Writer

## Losses
tf.summary.scalar("Loss", loss)

## Reward mean
tf.summary.scalar("Reward_mean", mean_reward_)

merged_summary_op = tf.summary.merge_all() #procedure d'affichage groupée dans tensorboard

Instructions for updating:
Use keras.layers.Dense instead.


2021-08-04 15:49:28,320  WARNING: From <ipython-input-4-ed69ceeb7041>:53: dense (from tensorflow.python.layers.core) is deprecated and will be removed in a future version.
Instructions for updating:
Use keras.layers.Dense instead.


Instructions for updating:
Please use `layer.__call__` method instead.


2021-08-04 15:49:28,338  WARNING: From /home/saadubuntu/Documents/Teresa_robot_internship_2021/venv/lib/python3.6/site-packages/tensorflow_core/python/layers/core.py:187: Layer.apply (from tensorflow.python.keras.engine.base_layer) is deprecated and will be removed in a future version.
Instructions for updating:
Please use `layer.__call__` method instead.


## Training

In [5]:
# Parameters for the training
max_episodes = 2
gamma = 0.95 # Discount rate
#max_batch = NbStat*5
max_batch = 20
episodes_succeded = 0

client.run() # This run the main loop of ROS
teresa_controller = Teresa(client) # Robot API
env = RobotEnv(teresa_controller, client) # Training Environment

env.reset() # Restarting the environment to the initial state
write_op = tf.summary.merge_all()
allRewards = []
total_rewards = 0
maximumRewardRecorded = 0
episode = 0
episode_states, episode_actions, episode_rewards = [],[],[]

saver = tf.train.Saver()
NbStat = state_size

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    
    #writer = tf.summary.FileWriter("./tensorboard/pg/1",sess.graph)
     
    for episode in range(max_episodes):
        
        episode_rewards_sum = 0

        # Launch the game
        state = env.reset()
        print(NbStat)
        ne_state=np.identity(NbStat)[state:state+1]
        episode_length=0
        while True:
            episode_length+=1
            if episode_length > max_batch:
                print ("tooooooooo long")
                break      
            # Choose action a, remember WE'RE NOT IN A DETERMINISTIC ENVIRONMENT, WE'RE OUTPUT PROBABILITIES.
            #state=int(state)
            #print("state",state,NbStat)
            ne_state=np.identity(NbStat)[state:state+1]
            action_probability_distribution = sess.run(action_distribution, feed_dict={input_: ne_state.reshape([1,NbStat])})
            action = np.random.choice(range(action_probability_distribution.shape[1]), p=action_probability_distribution.ravel())  # select action w.r.t the actions prob

            # Perform a
            ext=False
            #print("in actor mstep",state,"real ibn self")
            new_state, reward, done, info = env.step(action)
            env.render()
            #print("after action state",new_state,NbStat)
            #print(" drz actor mstep",state,"real ibn self",new_state,"rew",reward,"done",done)
            # Store s, a, r
            episode_states.append(ne_state)
                        
            # For actions because we output only one (the index) we need 2 (1 is for the action taken)
            # We need [0., 1.] (if we take right) not just the index
            action_ = np.zeros((action_size), dtype=int)
            action_[action] = 1
            
            #print("action proba",action_probability_distribution,"st",state,"new",new_state)
            state = new_state
            episode_actions.append(action_)
            
            episode_rewards.append(reward)
            if done:
                # Calculate sum reward
                if reward == 1:
                    episodes_succeded += 1
                
                episode_rewards_sum = np.sum(episode_rewards)/episode_length  # HA addded the sum 
                print(action_probability_distribution,"action proba",episode_rewards_sum,"length",episode_length,"rew",reward)
                allRewards.append(episode_rewards_sum)
                
                total_rewards = np.sum(allRewards)
                
                # Mean reward
                mean_reward = np.divide(total_rewards, episode+1)
                
                
                maximumRewardRecorded = np.amax(allRewards)
                
                print("==========================================")
                print("Episode: ", episode,"length",episode_length)
                print("Reward: ", episode_rewards_sum)
                print("Mean Reward","val",mean_reward)
                print("Max reward so far: ", maximumRewardRecorded)
                
                # Calculate discounted reward
                discounted_episode_rewards = discount_and_normalize_rewards(episode_rewards)
                
                #print("disco",discounted_episode_rewards)               
                # Feedforward, gradient and backpropagation
                #loss_, _ = sess.run([loss, train_opt], feed_dict={input_: np.vstack(np.array(episode_states)),
                #                                                 actions: np.vstack(np.array(episode_actions)),
                #                                                 discounted_episode_rewards_: discounted_episode_rewards 
                #                                                })
                loss_, _ = sess.run([loss, train_opt], feed_dict={input_: np.vstack(np.array(episode_states)),
                                                                 actions: np.vstack(np.array(episode_actions)),
                                                                 discounted_episode_rewards_: discounted_episode_rewards
                                                                })
                
 
                                                                 
                # Write TF Summaries
                summary = sess.run(write_op, feed_dict={input_: np.vstack(np.array(episode_states)),actions: np.vstack(np.array(episode_actions)),discounted_episode_rewards_: discounted_episode_rewards,
                                                                    mean_reward_: mean_reward  })
                
               
#                 writer.add_summary(summary, episode)
#                 writer.flush()
                
            
                
                # Reset the transition stores
                episode_states, episode_actions, episode_rewards = [],[],[]
                
                break
            
            
            
        # Save Model
    saver.save(sess, "pgpendul.ckpt")
    print("Model saved")
    print(episodes_succeded / max_episodes)

3596
[[0.25010297 0.25055847 0.24983136 0.24950722]] action proba 0.5 length 2 rew 1
Episode:  0 length 2
Reward:  0.5
Mean Reward val 0.5
Max reward so far:  0.5
3596
[[0.2513268  0.24580231 0.25077066 0.25210026]] action proba 0.5 length 2 rew 1
Episode:  1 length 2
Reward:  0.5
Mean Reward val 0.5
Max reward so far:  0.5
Model saved
1.0


In [6]:
print(episodes_succeded)

2


## Testing the Neural Network

### Structure of the NN

In [11]:
 
NbStat = NB_STATES
state_size = NbStat
action_size = 4
# new_graph = tf.Graph()
initializer=tf.initializers.glorot_uniform()
learning_rate = 0.01

def discount_correct_rewards(r, gamma=0.99):
  """ take 1D float array of rewards and compute discounted reward """
  discounted_r = np.zeros_like(r)
  running_add = 0
  for t in reversed(range(0, r.size)):
    #if r[t] != 0: running_add = 0 # reset the sum, since this was a game boundary (pong specific!)
    running_add = running_add * gamma + r[t]
    discounted_r[t] = running_add

  discounted_r -= discounted_r.mean()
  discounted_r /- discounted_r.std()
  return discounted_r

# Setup TensorBoard Writer
def discount_and_normalize_rewards(episode_rewards):
    discounted_episode_rewards = np.zeros_like(episode_rewards)
    cumulative = 0.0
    #print("len episode rewards",episode_rewards)
    for i in reversed(range(len(episode_rewards))):
        cumulative = cumulative * gamma + episode_rewards[i]
        #print("dans boucle",episode_rewards[i],"cyl",cumulative)
        discounted_episode_rewards[i] = cumulative
    
    mean = np.mean(discounted_episode_rewards)
    std = np.std(discounted_episode_rewards)
    if std :
        discounted_episode_rewards = (discounted_episode_rewards - mean) / (std)
    else:
        discounted_episode=[]
        discounted_episode_rewards[0] = np.array(mean)
        print("ATTTTTTTTTTTTTTTTTT")
    #print("dis",discounted_episode_rewards,"std",std)
    
    return discounted_episode_rewards
def discount_and_normalize_rewards(episode_rewards):
    discounted_episode_rewards = np.zeros_like(episode_rewards)
    cumulative = 0.0
    #print("len episode rewards",episode_rewards)
    for i in reversed(range(len(episode_rewards))):
        cumulative = cumulative * gamma + episode_rewards[i]
        #print("dans boucle",episode_rewards[i],"cyl",cumulative)
        discounted_episode_rewards[i] = cumulative
    
    mean = np.mean(discounted_episode_rewards)
    std = np.std(discounted_episode_rewards)
    if std :
        discounted_episode_rewards = (discounted_episode_rewards - mean) / (std)
    else:
        discounted_episode=[]
        discounted_episode_rewards[0] = np.array(mean)
        print("ATTTTTTTTTTTTTTTTTT")
    #print("dis",discounted_episode_rewards,"std",std)
    
    return discounted_episode_rewards

with tf.name_scope("inputs"):
    input_ = tf.placeholder(tf.float32, [None, state_size], name="input_")
    actions = tf.placeholder(tf.int32, [None, action_size], name="actions")
    discounted_episode_rewards_ = tf.placeholder(tf.float32, [None,], name="discounted_episode_rewards")
    
    # Add this placeholder for having this variable in tensorboard
    mean_reward_ = tf.placeholder(tf.float32 , name="mean_reward")

    with tf.name_scope("fc1"):
        fc1 = tf.layers.dense(input_ , 20, activation=tf.nn.relu,kernel_initializer=initializer)

    with tf.name_scope("fc2"):
        fc2 = tf.layers.dense(fc1, action_size,activation= tf.nn.relu, kernel_initializer=initializer)
    
    with tf.name_scope("fc3"):
        fc3 = tf.layers.dense(fc2, action_size, activation= None,kernel_initializer=initializer)

    with tf.name_scope("softmax"):
        action_distribution = tf.nn.softmax(fc3)

    with tf.name_scope("loss"):
        # tf.nn.softmax_cross_entropy_with_logits computes the cross entropy of the result after applying the softmax function
        # If you have single-class labels, where an object can only belong to one class, you might now consider using 
        # tf.nn.sparse_softmax_cross_entropy_with_logits so that you don't have to convert your labels to a dense one-hot array. 
        neg_log_prob = tf.nn.softmax_cross_entropy_with_logits_v2(logits = fc3, labels = actions)
        #loss = tf.nn.sparse_softmax_cross_entropy_with_logits (neg_log_prob * discounted_episode_rewards_)
        loss = tf.reduce_mean(neg_log_prob * discounted_episode_rewards_) 
        
    
    with tf.name_scope("test"):
        train_opt = tf.train.AdamOptimizer(learning_rate).minimize(loss)


## Losses
## TRAINING Hyperparameters

# tf.summary.scalar("Loss", loss)

# ## Reward mean
# tf.summary.scalar("Reward_mean", mean_reward_)

max_episodes = 500

gamma = 0.95 # Discount rate
max_batch = NbStat*5
    
episode_rewards_sum = 0

        # Launch the game
    #state = env.reset()
    #ne_state=np.identity(NbStat)[state:state+1]
    #env.render()
episode_length=0

### Running the NN

In [12]:

saver = tf.train.Saver()

client.run() # This run the main loop of ROS
teresa_controller = Teresa(client) # Robot API
env = RobotEnv(teresa_controller, client) # Training Environment

with tf.Session() as sess:
    # sess.run(tf.global_variables_initializer())
   
        # Load the model
    print(saver.restore(sess, "pgpendul.ckpt"))
    if not saver.restore(sess, "pgpendul.ckpt"):
        print('file not found')
    total_rewards = 0
    for episode in range(50):
        state = env.reset()
        #ne_state=np.identity(NbStat)[state:state+1]
        step = 0
        done = False
        
        print("****************************************************")
        print("EPISODE ", episode)

       
        #while True:
        j = 0
        #The Q-Network
        while j < 500:
            j+=1
            state=int(state)
            ne_state=np.identity(NbStat)[state:state+1]
            # Choose action a, remember WE'RE NOT IN A DETERMINISTIC ENVIRONMENT, WE'RE OUTPUT PROBABILITIES.
            action_probability_distribution = sess.run(action_distribution, feed_dict={input_: ne_state.reshape([1,NbStat])})
            print(action_probability_distribution)
            action = np.random.choice(range(action_probability_distribution.shape[1]), p=action_probability_distribution.ravel())  # select action w.r.t the actions prob
            #action = np.argmax(action_probability_distribution)
            

            # new_state, reward, done, info = env.step(int(action),True)
            new_state, reward, done, info = env.step(int(action))

            print("state",state,"ne_state",new_state,"action",action) 
            total_rewards += reward
            env.render()
            if done:    
                #rewards.append(total_rewards)
                print ("Score", total_rewards)
                break
            state = new_state
    env.close()
print ("Score over time: " ,  total_rewards)

INFO:tensorflow:Restoring parameters from pgpendul.ckpt


2021-08-04 15:41:57,096     INFO: Restoring parameters from pgpendul.ckpt


NotFoundError: Restoring from checkpoint failed. This is most likely due to a Variable name or other graph key that is missing from the checkpoint. Please ensure that you have not altered the graph expected based on the checkpoint. Original error:

Key dense_3/bias not found in checkpoint
	 [[node save_2/RestoreV2 (defined at /home/saadubuntu/Documents/Teresa_robot_internship_2021/venv/lib/python3.6/site-packages/tensorflow_core/python/framework/ops.py:1748) ]]

Original stack trace for 'save_2/RestoreV2':
  File "/home/saadubuntu/Documents/Teresa_robot_internship_2021/venv/lib/python3.6/runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "/home/saadubuntu/Documents/Teresa_robot_internship_2021/venv/lib/python3.6/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/home/saadubuntu/Documents/Teresa_robot_internship_2021/venv/lib/python3.6/site-packages/ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "/home/saadu